In [2]:
import json
import numpy as np
import pandas as pd
from elasticsearch import Elasticsearch
from bokeh.charts import Bar, output_file, show

es = Elasticsearch([{'host':'localhost','port':9200}])
#print json.dumps(es.info(),indent=2)

In [3]:
response = es.search(
    index="twitter2",
    body={
            "query": {
                "term": {
                    "text": "exploit"
                }
            },
            "from": 0,
            "size":1000
        }
)
#print json.dumps(response,indent=2)

ConnectionTimeout: ConnectionTimeout caused by - ReadTimeoutError(HTTPConnectionPool(host='localhost', port=9200): Read timed out. (read timeout=10))

In [ ]:
response = es.search(
    index="twitter2",
    body={
          "query": {
              "range": {
                  "timestamp": {
                      "gt": "2000-10-01T15:39:11.000Z",
                      "lt": "2016-10-20T15:39:13.000Z"
                  }
              }
          },
          "from": 0,
          "size": 1000
    }
)
#print json.dumps(response,indent=2)

In [ ]:
response = es.search(
    index="twitter2",
    body={
          "query": {
            "bool": {
              "must": [
                {
                  "range": {
                    "timestamp": {
                      "gt": "2000-10-02T15:39:11.000Z",
                      "lt": "2016-10-20T16:39:13.000Z"
                    }
                  }
                },
                {
                  "term": {
                    "text": "exploit"
                  }
                }
              ],
              "must_not": [],
              "should": []
            }
          },
          "from": 0,
          "size": 1000
        }
)
# for tweet in response['hits']['hits']:
#    print tweet['_source']['user']
#    print tweet['_source']['text']
#    print tweet['_source']['timestamp']
#    print "\n"


In [17]:
response = es.search(
    index="twitter2",
    body={
        "query": {
            "term": {
                "text": "exploits"
            }
        },
        "size" : 0,
        "aggs": {
            "time": {
                "date_histogram": {
                    "field": "tweet.created_at",
                    "interval":"month"
              },
            "aggs":{
                    "name":{
                        "terms":{
                            "field":"tweet.uid"
                        }
                    }
                }
            }
        }
    }
)

#print json.dumps(response,indent=2)
#for user in response['aggregations']['time']['buckets']:
#    print str(user['doc_count']) + "\t" + str(user['key_as_string'])
#    print str(user['doc_count']) + "\t" + str(str(user['key_as_string']).split(' ')[:3])
    

INFO:elasticsearch:GET http://localhost:9200/twitter2/_search [status:200 request:0.022s]


In [25]:
# print type(response)
# print response
#print response["hits"]
df = pd.read_json(json.dumps(response['aggregations']['time']['buckets']))
pd.options.display.max_rows = 4000
pd.options.display.max_columns = 200
output = df[["doc_count","key_as_string"]]
#print(output)

In [26]:
p = Bar(output, 'key_as_string', values='doc_count', title="Total Tweets by month")
output_file("bar.html")
show(p)


INFO:bokeh.core.state:Session output file 'bar.html' already exists, will be overwritten.
